In [1]:
%matplotlib inline
from vj_processing import *
from IPython.display import clear_output
#from utilities.containers import *
#from utilities.ccd import CCD
#pixmapfile = '../data/PixelMaps/Storage/PixelMaps/K2_target_pixels.hdf5'
pixmapfile = '../K2PixelMap_Camp8_Mod6_Chan2.hdf5'

In [2]:
#get_lc_url(EPIC, 8)
K2FOV.Field.get_channels(16)
from matplotlib.pyplot import savefig

https://archive.stsci.edu/missions/k2/lightcurves/c8/220200000/89355/ktwo220289355-c08_llc.fits
https://archive.stsci.edu/missions/k2/lightcurves/c8/220200000/89000/ktwo220289355-c08_llc.fits

In [3]:
cmap = palettable.colorbrewer.diverging.PuOr_11.mpl_colormap

In [4]:
def set_tick_sizes(ax):
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(26) 

    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(26)

def get_sf(arr):
    
    sf = []
    with LoadingBar(True) as l:
        for i in xrange(len(arr)):
            sf.append(np.var([arr[j:j+i] for j in xrange(i, len(arr)-i)]))
            l.update_bar(1.0*i/len(arr))
    return sf

def plot_pixel_sf(args, title=None, ylim=None, STEP=120, smooth=False):
    
    fig, ax = subplots(1,1, figsize=(16,9*3))
    args = list(args)
    
    image = np.array([y for t,y,_,b in args][::-1])
    image = np.array([get_sf(line) for line in image])
    
    #print image
    if smooth: # This is very very slow
        if smooth is True:
            smooth = 30
        #image[::] = np.array([runningMean(y,120) for y in image])
        image = np.array(runningMean(image.T,smooth,axis=0)).T
    if STEP > 1:
        image = np.array([np.mean(image[:,i:i+STEP], axis=1) for i in xrange(0,image.shape[1],STEP)]).T
        
    col, row = image.shape
    t = 0.02043229*np.arange(N)+2559.06849083

    #cmap = rplot.palettable.colorbrewer.diverging.BrBG_11.mpl_colormap
    #cmap = rplot.palettable.cubehelix.perceptual_rainbow_16.mpl_colormap
    cmap = palettable.cubehelix.perceptual_rainbow_16.mpl_colormap
    if ylim is not None:
        im = ax.imshow(image, cmap=cmap, vmin=ylim[0], vmax=ylim[1], aspect='auto')
        cbar=fig.colorbar(im, ticks=np.linspace(ylim[0],ylim[1], 1+(ylim[1]-ylim[0])))
    else:
        im = ax.imshow(image, cmap=cmap, aspect='auto')
        cbar = fig.colorbar(im)
    if title is not None:
        ax.set_title(title)
    

        
    ax.set_yticks(np.linspace(0, ax.get_ybound()[1]+1, 20))
    
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: int((col-1-y)*100/col+0.5)))
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: int(x*1.0/row*N*0.02043229+2559.06849083)))
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Percentile')
    #print ax.get_ylim()
    
    if args[0][3] is not None:
        #bins = np.array([bins for _,_,_,bins in args][::-1])
        means = np.array([m for m in _, y, _, _ in args ])
        #print bins.shape
        #axis_labels = ["{:.2f}".format(i) for i in fluxToMag(np.mean(bins, axis=1))[::-1]]
        axis_labels = ["{:.2f}".format(i) for i in fluxToMag(np.mean(means, axis=1))]
        #print axis_labels
        #ax.tick_params(labelbottom='on',labeltop='off', labelleft="on", 
        #    top='off', left='on', right='off')
        #ax2 = fig.add_axes(ax.get_position(), frameon=False)
        ax2 = ax.twinx()
        ax2.set_yticks(np.linspace(0, ax2.get_ybound()[1]+1, 20))
        #print axis_labels
        ax2.set_yticklabels(axis_labels[int(i+0.5)] for i in np.linspace(0, len(axis_labels), 20, endpoint=False))
        #ax2.tick_params(labelbottom='off',labeltop='off', labelleft="off", labelright='on',
        #    bottom='off', left='off', right='on', which='both')
        
        #ax2.set_ylim(ax.get_ylim())
        #ax2.set_xlim(ax.get_xlim())
        #plt.draw()
        #ax2.set_position(ax.get_position())
        #plt.draw()
        #print axis_labels
        ax.autoscale(False)
        ax2.autoscale(False)
        def formatter(x, pos):
            #print x
            if pos is None:
                return ''
            idx = int(x*len(axis_labels)/(ax2.get_ybound()[1]+1)+0.5)
            if idx >= len(axis_labels) or idx < 0:
                return ''
            if np.isnan(axis_labels[idx]):
                return 'NaN'
            #print "{:.2f}".format(axis_labels[idx])
            return "{:.1f}".format(axis_labels[idx])
        #ax2.yaxis.set_major_formatter(ticker.FuncFormatter(formatter))
        #ax2.yaxis.set_label_position("right")
        #ax2.set_ylim(ax.get_ylim())
        #ax2.set_ylim(axis_labels[0], axis_labels[-1])
        #ax2.yaxis.set_view_interval(axis_labels[0], axis_labels[-1])
        #ax2.set_adjustable('box-forced')
        #plt.draw()
        
        #print "ax2:",ax2.get_ylim()
        #print "ax: ",ax.get_ylim()
        #ax2.set_ylabel('Kep Mag')
    
    return image


def plot_pixel_image(args, title=None, ylim=None, STEP=120, smooth=False):
    
    
    import matplotlib.pyplot as plt
    fig, ax = subplots(1,1, figsize=(16,9))
    args = list(args)
    
    image = np.array([y for t,y,_,b in args][::-1])
    N = image.shape[1]
    
    #print image
    if smooth: # This is very very slow
        if smooth is True:
            smooth = 30
        #image[::] = np.array([runningMean(y,120) for y in image])
        image = np.array(runningMean(image.T,smooth,axis=0)).T
    if STEP > 1:
        image = np.array([np.mean(image[:,i:i+STEP], axis=1) for i in xrange(0,image.shape[1],STEP)]).T
        
    col, row = image.shape
    t = 0.02043229*np.arange(N)+2559.06849083

    #cmap = rplot.palettable.colorbrewer.diverging.BrBG_11.mpl_colormap
    #cmap = rplot.palettable.cubehelix.perceptual_rainbow_16.mpl_colormap
    #cmap = rplot.palettable.colorbrewer.diverging.PuOr_11.mpl_colormap
    cmap = palettable.colorbrewer.diverging.PuOr_11.mpl_colormap
    if ylim is not None:
        im = ax.imshow(image, cmap=cmap, vmin=ylim[0], vmax=ylim[1], aspect='auto')
        cbar=fig.colorbar(im, ticks=np.linspace(ylim[0],ylim[1], 1+(ylim[1]-ylim[0])), pad=0.1)
    else:
        im = ax.imshow(image, cmap=cmap, aspect='auto')
        cbar = fig.colorbar(im, pad=0.16)
        
    if title is not None:
        ax.set_title(title, fontsize=30)
    ax.plot(0, 0, "-", label=title[-5:])    
    ax.legend(fontsize = 50, frameon=False, loc = 8, handlelength=0, handletextpad=0,)

        
    ax.set_yticks(np.linspace(0, ax.get_ybound()[1]+1, 10))
    
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: int((col-1-y)*100/col+0.5)))
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: int(x*1.0/row*N*0.02043229+2559.06849083)))
    
    ax.set_xlabel('Date', fontsize=24)
    ax.set_ylabel('Percentile', fontsize=24)
    #print ax.get_ylim()
    
    if args[0][3] is not None:
        bins = np.array([bins for _,_,_,bins in args][::-1])
        #print bins.shape
        axis_labels = ["{:.2f}".format(i) for i in fluxToMag(np.mean(bins, axis=1))[::-1]]
        #print axis_labels
        #ax.tick_params(labelbottom='on',labeltop='off', labelleft="on", 
        #    top='off', left='on', right='off')
        #ax2 = fig.add_axes(ax.get_position(), frameon=False)
        ax2 = ax.twinx()
        ax2.set_yticks(np.linspace(0, ax2.get_ybound()[1]+1, 10))
        #print axis_labels
        ax2.set_yticklabels([axis_labels[int(i+0.5)] for i in np.linspace(0, len(axis_labels), 10, endpoint=False)], {'fontsize':26})
        #ax2.tick_params(labelbottom='off',labeltop='off', labelleft="off", labelright='on',
        #    bottom='off', left='off', right='on', which='both')
        
        #ax2.set_ylim(ax.get_ylim())
        #ax2.set_xlim(ax.get_xlim())
        #plt.draw()
        #ax2.set_position(ax.get_position())
        #plt.draw()
        #print axis_labels
        ax.autoscale(False)
        ax2.autoscale(False)
        def formatter(x, pos):
            #print x
            if pos is None:
                return ''
            idx = int(x*len(axis_labels)/(ax2.get_ybound()[1]+1)+0.5)
            if idx >= len(axis_labels) or idx < 0:
                return ''
            if np.isnan(axis_labels[idx]):
                return 'NaN'
            #print "{:.2f}".format(axis_labels[idx])
            return "{:.1f}".format(axis_labels[idx])
        #ax2.yaxis.set_major_formatter(ticker.FuncFormatter(formatter))
        #ax2.yaxis.set_label_position("right")
        #ax2.set_ylim(ax.get_ylim())
        #ax2.set_ylim(axis_labels[0], axis_labels[-1])
        #ax2.yaxis.set_view_interval(axis_labels[0], axis_labels[-1])
        #ax2.set_adjustable('box-forced')
        #plt.draw()
        
        #print "ax2:",ax2.get_ylim()
        #print "ax: ",ax.get_ylim()
        #ax2.set_ylabel('Kep Mag')
        
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(24) 

    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(24) 
        
    for tick in ax2.xaxis.get_major_ticks():
        tick.label.set_fontsize(24) 

    for tick in ax2.yaxis.get_major_ticks():
        tick.label.set_fontsize(24) 
    
   
    return image

def make_pixel_lc(lc, percentiles, stat='median', diff=False, med_norm=False):

    N = lc.shape[0]
    t = 0.02043229*np.arange(N)+2559.06849083
    bins = np.array(zip(percentiles[:-1], percentiles[1:]))
    bins = filter(lambda x: None not in x, bins) # for percentiles containing Nones
    idx = {'var':2, 'median':3, 'mean':4}[stat]
    stat = np.empty((2,)) # so we can keep track of the median magnitudes between percentiles
    y = np.empty((N,))
    
    for i,b in enumerate(bins):
        y[::] = lc[:,i,idx]
        stat[::] = [np.nanmedian(lc[:,i,0]),np.nanmedian(lc[:,i,1])]
        
        if diff and i:
            y[::] -= lc[:,i-1,idx]
            stat[::] = np.nanmean([stat,[np.nanmedian(lc[:,i-1,0]),np.nanmedian(lc[:,i-1,1])]], axis=0)

        if med_norm: # divide out by mean
            y[::] /= np.nanmedian(y)
        else:
            med = np.nanmedian(y)
            std = np.nanstd(y)
            y[::] = (y-med)/std
        
        label = '{:.2f}\% to {:.2f}\%, KepMag: {:.2f} to {:.2f}'.format(*(tuple(b)+tuple(fluxToMag(stat))))
        yield t,copy(y),label,copy(stat)
    

In [5]:
def generate_lightcurve(pixels, aperture, mednorm=False, bkg_correct=True):
    
    lc = np.empty(len(pixels))
    if bkg_correct:
        lc[::] = np.nansum(pixels[:,aperture] - np.nanmedian(pixels[:,~aperture], axis=1).reshape(-1,1), axis=1)
    else:
        lc[::] = np.nansum(pixels[:,aperture], axis=1)
    #lc[::] = np.nansum(pixels[:,aperture], axis=1) - np.nanmedian(pixels[:,~aperture], axis=1)
    if mednorm:
        lc[::] /= np.nanmedian(pixels[:, aperture], axis=1)
    return lc

def get_full_flux_pixels(pipeline):
    d = pipeline.pix['TARGETTABLES'].data
    return d['FLUX'] + d['FLUX_BKG']

def get_aperture(pipeline, thresh):
    return np.array(pipeline.vj['APERTURE'].data > thresh, dtype=np.bool)

def print_aperture(*aperture, **kwargs):
    imgs = [print_image(ap, show=False, **kwargs) for ap in aperture]
    for ims in zip(*imgs):
        print u''.join(ims)
        
def print_images(*images, **kwargs):
    title = ["",""]
    if 'title' in kwargs:
        if type(kwargs['title']) is not str:
            title = kwargs['title']
        else:
            title[0] = kwargs['title']
        del kwargs['title']
    imgs = [print_image(im, show=False, title=t, **kwargs) for im,t in zip(images, title)]
    for ims in zip(*imgs):
        print u''.join(ims)
    
def get_vj_aperture(EPIC, aperture_type, aperture_index):
    
    cache_epics = dict(get_vj_aperture.cache)
    if EPIC in cache_epics:
        return cache_epics[EPIC][aperture_type].data[aperture_index].astype(np.bool)
    url = get_vj_url(EPIC, 8)
    hdu = next(fits_downloader([url]))
    get_vj_aperture.cache.append((EPIC,hdu))
    return get_vj_aperture(EPIC, aperture_type, aperture_index)

get_vj_aperture.cache = []
  
    
def print_image(image, vmin=None, vmax=None, scale='linear', size=1, invert=False, cbar=False, title=None, blank_nan=False, show=True, color='w'):
    
    #text_color = {'red':u'\u001b[31m', 'green':u"\u001b[32m", 'blue':u"\u001b[34m"}
    pixel_images = [u'█',u'▓',u'▒',u'░',u' ']
    #pixel_images = [text_color[key]+i for i in pixel_images for key in ['green', 'blue','red']]
    
    ###
    pixel_images = create_mono_bkg('k') + create_gradient(*('k'+color))[1:-1] + create_mono(color)
    ####
    N_unique = len(set(image.ravel()))
    if N_unique < len(pixel_images):
        pixel_images = [pixel_images[int(i)] for i in np.linspace(0,len(pixel_images)-0.1, N_unique)]
    
    N_levels = len(pixel_images)-1
    nan_char = [u'╳']
    if invert:
        pixel_images = pixel_images[::-1]
    if blank_nan:
        nan_char = [pixel_images[0]]
    pixel_images = nan_char + pixel_images
    
    scaled_image = copy(image).astype(np.float)
    mask = ~np.isnan(scaled_image)
    
    if scale == 'log':
        scaled_image[mask] = np.log10(scaled_image[mask])
        mask = ~np.isnan(scaled_image) # in case of new NaNs
    elif scale == 'symlog':
        scaled_image[mask] = np.log10(scaled_image[mask]*np.sign(scaled_image[mask]))*np.sign(scaled_image[mask])
        mask = ~np.isnan(scaled_image) # in case of new NaNs
    if vmin is None:
        vmin = np.nanmin(scaled_image)
    if vmax is None:
        vmax = np.nanmax(scaled_image)
        if vmax == vmin:
            vmin -= 1
    scaled_image[mask] -= vmin
    scaled_image[mask] /= (vmax-vmin)/(N_levels)
    #print scaled_image

    scaled_image[scaled_image > N_levels] = N_levels
    scaled_image[scaled_image < 0] = 0
    scaled_image[~mask] = -1
    scaled_image += 1
    
    #print scaled_image, N_levels
    image = []
    if cbar:
        values = np.linspace(vmin, vmax, N_levels+1)
        if scale == 'log':
            values = np.logspace(vmin, vmax, N_levels+1)
        elif scale == 'symlog':
            values = np.concatenate((-np.logspace(0, -vmin, (N_levels+1)/2)[::-1], np.logspace(0, vmax, (N_levels+1)/2)))
        def cbar():
            try:
                n_rows = scaled_image.shape[0]
                interval = max((n_rows*size+size/2)/(len(values)-1),1)
                for i in xrange(size/2,n_rows*size+size/2):
                    #index = i/interval
                    index = i
                    index = int(index*(len(pixel_images)-1)/(n_rows*size+size/2))
                    #if index == N_levels+1:
                    #    index = N_levels
                    if not (i - interval/2 + 1) % interval:
                        yield u"▕{0}{0}▏─ {1:.2f}".format(pixel_images[index+1], values[index])
                    else:
                        yield u"▕{0}{0}▏".format(pixel_images[index+1])
            except IndexError:
                print i, index, interval, len(values), N_levels, size, n_rows*size+size/2, n_rows
                raise
    else:
        def cbar():
            for i in xrange(scaled_image.shape[0]*size):
                yield ''
    color_bar = cbar()
    if title is not None:
        extra = 11 if cbar else 0
        padding = ' '*max(((scaled_image.shape[1]*size*2+extra-len(title))/2),0)
        image.append(''.join((padding, title, padding)))
    
    
    for i, row in enumerate(scaled_image):
        for _ in xrange(size):
            try:
                image.append(u'▕'+u''.join((pixel_images[int(r)]*2*size for r in row))+u'▏'+next(color_bar)+u' ')
            except IndexError:
                print row, N_levels, mask[i], i
                print scaled_image, mask
                raise
                
    if show:            
        print u'\n'.join(image)
    return image
                
pixel_images = [u'█',u'▓',u'▒',u'░',u' ']
text_color = {key:u'\u001b[3%im' % i for i, key in enumerate('krgybmcw')}
text_color.update({'_':u'\u001b[0m'})
background_color = {key:u'\u001b[4%im' % i for i, key in enumerate('krgybmcw')}

bc = background_color
tc = text_color
pim = pixel_images

def create_mono_bkg(c):
    return [tc['_']+bc[c]+p+tc['_'] for p in pim]

def create_mono(c):
    return [tc['_']+tc[c]+p+tc['_'] for p in pim]

def _create_gradient(c1, c2):

    c = (c1,c2)
    return [tc[c[i%2]]+bc[c[1-(i%2)]]+pim[i/2 if not i%2 else 3-i/2]+tc['_'] for i in xrange(8)]

def create_gradient(*args):
    
    return sum([_create_gradient(c1, c2)[1 if c1 != args[0] else 0:None if c2 == args[-1] else -1] for c1, c2 in zip(args[:-1], args[1:])], [])#+[tc['_']+u' '+tc['_']]

def fold(x, step):
    
    x_ = np.nanmean(filter(lambda y: len(y)==step, [x[i:i+step:] for i in xrange(0,len(x), step)]), axis=0)
    return x_

# STEP=1
STEP = N/len(percentiles)*(9)/16
SMOOTH=120

title_args = (ccd.campaign, ccd.module, ccd.channel, STEP, SMOOTH)
title = "Campaign {}, Module {}, Channel {}, Step {}, Smooth {}, Stat {{}}, Display {{}}".format(*title_args)

args = make_pixel_lc(lc, percentiles, diff=False)

plot_pixel_image(args, ylim=(-1.0, 1.0), STEP=STEP, smooth=SMOOTH, title="Median Pixel Light Curves");

args = make_pixel_lc(lc, percentiles, diff=True)

plot_pixel_image(args, ylim=(-1.0, 1.0), STEP=STEP, smooth=SMOOTH, title="Differenced Median Pixel Light Curves");

args = make_pixel_lc(lc, percentiles, diff=False, stat='mean')

plot_pixel_image(args, ylim=(-0.5, 0.5), STEP=STEP, smooth=SMOOTH, title=title.format('Mean', 'Full'));

args = make_pixel_lc(lc, percentiles, diff=True, stat='mean')

plot_pixel_image(args, ylim=(-0.75, 0.75), STEP=STEP, smooth=SMOOTH, title=title.format('Mean', 'Difference'));

args = make_pixel_lc(lc, percentiles, diff=False, stat='var')

plot_pixel_image(args, ylim=(-0.75, 0.75), STEP=STEP, smooth=SMOOTH, title=title.format('Variance', 'Full'));

args = make_pixel_lc(lc, percentiles, diff=True, stat='var')

plot_pixel_image(args, ylim=(-0.75, 0.75), STEP=STEP, smooth=SMOOTH, title=title.format('Variance', 'Difference'));


In [7]:
#import richardsplot as rplot
#import palettable
cmap = rplot.palettable.colorbrewer.diverging.PuOr_11.mpl_colormap
#cmap = palettable.colorbrewer.diverging.PuOr_11.mpl_colormap

In [19]:
!pwd

/home/jackeline/Research/k2_pipeline/CARMA/notebooks


In [8]:
# ****ONLY RUN ONCE***COMMENT OUT AFTER**** make directory with correct path for saving pixel maps figures
!mkdir ../plots/c07

In [ ]:
# if you get an indexing error below edit file vj_processing.py line 70
#        for i,g in enumerate((gen.get_unordered(i) for i in xrange(N-1))):

In [8]:
pixmapfile = "/media/jackeline/UUI/PixelMaps/K2_target_pixels.hdf5"

#  Loop through

In [ ]:
percentiles = np.linspace(0, 100, 501)
STEP=1
STEP = N/len(percentiles)*(9)/16
SMOOTH=120

percentiles = np.linspace(0, 100, 501)

#edit campaign number below
campaign = 5
for module in range(0,25):
    for channel in xrange(4):
        ccd = CCD(module=module, channel=channel, field='FLUX', campaign=campaign)
        if os.path.isfile('../plots/c0%i/Module%iChannel%iPixelMapFull.png' % (ccd.campaign, ccd.module, ccd.channel)):
            continue

        if os.path.isfile('../plots/Module%iChannel%iPixelMapFull.png' % (ccd.module, ccd.channel)):
            continue
        try:
            cont = PixelMapContainer.from_hdf5(pixmapfile, ccd)
        except:
            try:
                print "Loading..."
                with LoadingBar(True) as bar:
                    cont = PixelMapContainer(ccd, bar=bar)
                    cont.save(pixmapfile, doc='Pixel values are represented by the \'FLUX\' data column of the K2 target pixel files')
            except Exception as e:
                    print "Could not load data for CCD: {}".format(ccd)
                    print e
                    continue
        gen = PixMapGenerator(cont, cache=True)
        print "Making", ccd

        N = len(gen.containers.containers[0].pixels)
        STEP = N/len(percentiles)*(9)/16
        
        lc = get_pixel_lc(gen, percentiles, mag_range=(1,50)) #Star

        args = make_pixel_lc(lc, percentiles, diff=False)
        plot_pixel_image(args, ylim=(-1.0, 1.0), STEP=STEP, smooth=SMOOTH, title="Median Pixel Light Curves M%i.%i" % (ccd.module, ccd.channel));
        savefig('../plots/c0%i/Module%iChannel%iPixelMapFull.png' % (ccd.campaign, ccd.module, ccd.channel))

        args = make_pixel_lc(lc, percentiles, diff=True)
        plot_pixel_image(args, ylim=(-1.0, 1.0), STEP=STEP, smooth=SMOOTH, title="Differenced Median Pixel Light Curves M%i.%i" % (ccd.module, ccd.channel));
        savefig('../plots/c0%i/Module%iChannel%iPixelMapDiff.png' % (ccd.campaign, ccd.module, ccd.channel))


Loading...

[############################################################-] Estimating...   
Could not load data for CCD: CCD(module=0, channel=0, field='FLUX', campaign=5)
list index out of range
[############################################################-] Estimating...Loading...


Could not load data for CCD: CCD(module=0, channel=1, field='FLUX', campaign=5)
list index out of range
Loading...


Could not load data for CCD: CCD(module=0, channel=2, field='FLUX', campaign=5)
list index out of range
Loading...

[############################################################-] Estimating...   
Could not load data for CCD: CCD(module=0, channel=3, field='FLUX', campaign=5)
list index out of range
Loading...


Could not load data for CCD: CCD(module=1, channel=0, field='FLUX', campaign=5)
list index out of range
Loading...


Could not load data for CCD: CCD(module=1, channel=1, field='FLUX', campaign=5)
list index out of range
Loading...


Could not load data for CCD: CCD(module=1, chann